In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy 
from spacy import displacy
from collections import Counter
import json
import random
from sklearn.model_selection import train_test_split

In [2]:
def rand_emot():
    e = ["(o_o)",":-)",":P",":D","x)","ᓚᘏᗢ","╯°□°）╯︵ ┻━┻",":)",
         "*<:-)","^_^","(⌐■_■)","¯\_(ツ)_/¯", "(T_T)",":o","OwO",
        "( ͡❛ ͜ʖ ͡❛)","(̶◉͛‿◉̶)","( ≖.≖)","(ㆆ_ㆆ)","ʕ•́ᴥ•̀ʔっ","( ◡́.◡̀)","(^◡^ )"]
    return random.choice(e)

with open('data/additional/scraped/data_30.json') as f:
    dataframe = pd.read_json(f)
    
train1 = dataframe.loc[(dataframe["Byline"] == "Magnus Boye")][:50]
train2 = dataframe.loc[(dataframe["Byline"] == "Morten Guldberg")][:50]
train3 = dataframe.loc[(dataframe["Byline"] == "Alexander Bitsch")][:50]
train4 = dataframe.loc[(dataframe["Byline"] == "Morten Bjerregaard")][:50]
train5 = dataframe.loc[(dataframe["Byline"] == "Silla Bakalus")][:50]


    
frames = [train1, train2,train3, train4, train5]
trainset = pd.concat(frames, sort=False)
bodies = list(trainset["Body"])
authors = list(trainset["Byline"])

train, test, train_labels, test_labels = train_test_split(bodies, authors, test_size=0.1, random_state=42)

In [3]:
# train_labels

In [4]:
dataframe.Byline.value_counts()

Silla Bakalus                445
Magnus Boye                   96
Morten Guldberg               85
Alexander Bitsch              65
Morten Bjerregaard            50
Tina Splidsboel               40
Christian Sejer Rasmussen     40
Jacob Nordestgaard Lage        1
Name: Byline, dtype: int64

In [5]:
#test strings
en_string = "This list has overlapping features with content features. For example, word n-grams will capture the content of the text along with stylometric tendencies. Content features consist of word frequencies, word and character n-grams, hapax legomena etc. This overlap is not of concern, however, as Sari et al. show, using content features is beneficial when performing authorship attribution of news articles because journalists often have certain topics they prefer writing about. They argue that using only stylometric features is beneficial when attributing authors to texts of the same topic or genre, e.g. law text or movie reviews."
da_test = 'Cecilia Lonning-Skovgaard har tidligere oplyst, at hun har søgt professionel hjælp til at forbedre sin ledelsesstil.En undersøgelse af det psykiske arbejdsmiljø i centralforvaltningen i Københavns kommune er "rystende læsning" og vidner om store svigt i toppen af ledelsen - herunder særligt beskæftigelses- og Integrationsborgmester Cecilia Lonning-Skovgaard (V). Sådan lyder det i et åbent brev fra en række fagforbund. Undersøgelsen dokumenterer en omfattende og fuldkommen uacceptabel, krænkende adfærd fra borgmesterens side og fjerner den sidste rest af tvivl om, hvor alvorlig og hvor uholdbar situationen er. Af undersøgelsen foretaget blandt 188 medarbejdere fremgår det, at 27 procent har oplevet krænkende adfærd, og 30 procent har været vidne til krænkende adfærd i centralforvaltningen. 47 procent af dem, der har været udsat for krænkelser og 68 procent af dem, der har været vidne til krænkende adfærd, svarer, at det er ”borgmesteren eller den øvrige politiske ledelse”, der står bag den krænkende adfærd.'

da_string = ['Cecilia Lonning-Skovgaard har tidligere oplyst, at hun har søgt professionel hjælp til at forbedre sin ledelsesstil.En undersøgelse af det psykiske arbejdsmiljø i centralforvaltningen i Københavns kommune er "rystende læsning" og vidner om store svigt i toppen af ledelsen - herunder særligt beskæftigelses- og Integrationsborgmester Cecilia Lonning-Skovgaard (V). Sådan lyder det i et åbent brev fra en række fagforbund. Undersøgelsen dokumenterer en omfattende og fuldkommen uacceptabel, krænkende adfærd fra borgmesterens side og fjerner den sidste rest af tvivl om, hvor alvorlig og hvor uholdbar situationen er. Af undersøgelsen foretaget blandt 188 medarbejdere fremgår det, at 27 procent har oplevet krænkende adfærd, og 30 procent har været vidne til krænkende adfærd i centralforvaltningen. 47 procent af dem, der har været udsat for krænkelser og 68 procent af dem, der har været vidne til krænkende adfærd, svarer, at det er ”borgmesteren eller den øvrige politiske ledelse”, der står bag den krænkende adfærd.']

train_corpus =['Til TV 2 oplyser Camilla Gregersen, formand for den akademiske fagforening DM, at Venstre bør overveje, om Cecilia Lonning-Skovgaard er den rette til posten.',
              '- Der er et kæmpe problem i forvaltningen, og det er centreret omkring den øverste ledelse. Det sender dårlig energi i hele systemet, og det er man nødt til at handle på nu.',
              'TV 2 har forsøgt at få en kommentar fra Venstres formand Jakob Ellemann-Jensen for at høre, hvordan han forholder sig til undersøgelsen og Cecilia Lonning-Skovgaards fremtid.']

test_corpus = ['I brevet fra fagforbundene lyder opfordringen desuden, at Beskæftigelses- og Integrationsudvalget i Københavns Kommune "øjeblikkeligt" skal få styr på det dårlige arbejdsmiljø under borgmesteren.',
               'TV 2 forsøger at få en kommentar fra Cecilia Lonning-Skovgaard.', 
               'Til Politiken siger hun:- Det tager jeg et kæmpe ansvar for, og det har jeg også sagt til medarbejderne her til morgen og undskyldt for. Jeg er i fuld gang med at arbejde på tonen over for medarbejderne, og jeg er også stoppet med at tage direkte kontakt til medarbejderne.',
               'Borgmesteren har tidligere oplyst, at hun har søgt professionel hjælp til at forbedre sin ledelsesstil.',
               'Undersøgelsen af det psykiske arbejdsmiljø blev igangsat i januar, efter at HK og DJØF i et åbent brev kritiserede Cecilia Lonning-Skovgaard for manglende indsigt i sin forvaltning og manglende forståelse for sin rolle som borgmester.']

In [6]:
def remove_punctuation(text):
    return re.sub(r',|\.|:|!|\?|;', '', text)

In [7]:
#POS tagger trained on Danish news and media corpus
POS_tagger_DK = spacy.load("da_core_news_md")

In [8]:
#Trains a TF-IDF vectorizer of word n-grams
def word_ngram_vectorizer(train_corpus, n): 
    
    vectorizer = TfidfVectorizer(max_features=3000, analyzer="word", ngram_range=(n,n))
    print("training vectorizer...",rand_emot())
    
    X = vectorizer.fit_transform(train_corpus)
    print("vectorizer fit!", rand_emot())
    
    ngrams = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=ngrams)
#     display(df.head(2))
      
    return X, vectorizer

#Trains a TF-IDF vectorizer of character n-grams
def char_ngram_vectorizer(train_corpus, n): 
    
    vectorizer = TfidfVectorizer(max_features=3000, analyzer="char", ngram_range=(n,n))
    print("training vectorizer...",rand_emot())
    
    X = vectorizer.fit_transform(train_corpus)
    print("vectorizer fit!", rand_emot())
    
    ngrams = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=ngrams)
#     display(df.head(2))
      
    return X, vectorizer

#Trains a TF-IDF vectorizer of POS n-grams. A POS corpus is generated in the function using a tagger for Danish
def POS_ngram_vectorizer(train_corpus, n): 
    
    #Create POS corpus
    POS_corpus = []

    for doc in train_corpus:
        tagged_doc = POS_tagger_DK(doc) #tag each document in corpus with POS tags using spacy
        POS_list = []

        for token in tagged_doc:
            POS_list.append(token.pos_)

        #concatenate as POS tags for the document
        POS_text = " ".join(POS_list)
        POS_corpus.append(POS_text)

    
    vectorizer = TfidfVectorizer(max_features=3000, analyzer="word", ngram_range=(n,n))
    print("training vectorizer...",rand_emot())
    
    X = vectorizer.fit_transform(POS_corpus)
    print("vectorizer fit!", rand_emot())
    
    ngrams = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=ngrams)
#     display(df.head(2))
    
    # Returns 
    return X, vectorizer

# Gets weights for terms based on trained vectorizer
# Works for both word and character ngrams
def get_tfidf_ngrams(vectorizer, test_corpus):
    '''Returns the TF-IDF weighted ngram frequencies of test documents'''
    #Multiple texts required
    return vectorizer.transform(test_corpus)

# Function generates POS test corpus first and then gets weights for terms based on trained vectorizer. 
def get_tfidf_POS_ngrams(vectorizer, test_corpus):
    '''Returns the TF-IDF weighted ngram frequencies of test documents'''
    #Create POS corpus
    POS_corpus = []

    for doc in test_corpus:
        tagged_doc = POS_tagger_DK(doc) #tag each document in corpus with POS tags using spacy
        POS_list = []

        for token in tagged_doc:
            POS_list.append(token.pos_)

        #concatenate POS tags as one string, i.e. the documented represented as the POS tags
        POS_text = " ".join(POS_list)
        POS_corpus.append(POS_text)
    
    #Multiple texts required
    return vectorizer.transform(POS_corpus)
    


In [9]:
X_unigrams_train,unigram_vectorizer = word_ngram_vectorizer(train,1)
X_char_unigrams_train,char_unigram_vectorizer = char_ngram_vectorizer(train,1)
X_POS_unigram_train, POS_unigram_vectorizer = POS_ngram_vectorizer(train,1)

X_bigrams_train,bigram_vectorizer = word_ngram_vectorizer(train,2)
X_char_bigrams_train,char_bigram_vectorizer = char_ngram_vectorizer(train,2)
X_POS_bigram_train, POS_bigram_vectorizer = POS_ngram_vectorizer(train,2)

X_trigrams_train,trigram_vectorizer = word_ngram_vectorizer(train,3)
X_char_trigrams_train,char_trigram_vectorizer = char_ngram_vectorizer(train,3)
X_POS_trigram_train, POS_trigram_vectorizer = POS_ngram_vectorizer(train,3)

training vectorizer... (o_o)
vectorizer fit! ^_^
training vectorizer... (ㆆ_ㆆ)
vectorizer fit! :D
training vectorizer... ( ≖.≖)
vectorizer fit! *<:-)
training vectorizer... (̶◉͛‿◉̶)
vectorizer fit! ( ͡❛ ͜ʖ ͡❛)
training vectorizer... :D
vectorizer fit! (o_o)
training vectorizer... ( ͡❛ ͜ʖ ͡❛)
vectorizer fit! :D
training vectorizer... :)
vectorizer fit! x)
training vectorizer... ^_^
vectorizer fit! ¯\_(ツ)_/¯
training vectorizer... :P
vectorizer fit! ( ◡́.◡̀)


In [10]:
X_unigrams_test = get_tfidf_ngrams(unigram_vectorizer, test).toarray()
X_char_unigrams_test = get_tfidf_ngrams(char_unigram_vectorizer, test).toarray()
X_POS_unigram_test = get_tfidf_POS_ngrams(POS_unigram_vectorizer, test).toarray()

X_bigrams_test = get_tfidf_ngrams(bigram_vectorizer, test).toarray()
X_char_bigrams_test = get_tfidf_ngrams(char_bigram_vectorizer, test).toarray()
X_POS_bigram_test = get_tfidf_POS_ngrams(POS_bigram_vectorizer, test).toarray()

X_trigrams_test = get_tfidf_ngrams(trigram_vectorizer, test).toarray()
X_char_trigrams_test = get_tfidf_ngrams(char_trigram_vectorizer, test).toarray()
X_POS_trigram_test = get_tfidf_POS_ngrams(POS_trigram_vectorizer, test).toarray()

In [11]:
from scipy.sparse import hstack

In [12]:
X_train = hstack((X_unigrams_train,X_bigrams_train))
X_train = hstack((X_train,X_trigrams_train))
X_train = hstack((X_train,X_char_unigrams_train))
X_train = hstack((X_train,X_char_bigrams_train))
X_train = hstack((X_train,X_char_trigrams_train))
X_train = hstack((X_train,X_POS_unigram_train))
X_train = hstack((X_train,X_POS_bigram_train))
X_train = hstack((X_train,X_POS_trigram_train))



In [13]:
X_test = np.hstack((X_unigrams_test,X_bigrams_test))
X_test = np.hstack((X_test,X_trigrams_test))
X_test = np.hstack((X_test,X_char_unigrams_test))
X_test = np.hstack((X_test,X_char_bigrams_test))
X_test = np.hstack((X_test,X_char_trigrams_test))
X_test = np.hstack((X_test,X_POS_unigram_test))
X_test = np.hstack((X_test,X_POS_bigram_test))
X_test = np.hstack((X_test,X_POS_trigram_test))


In [14]:
X_test.shape

(25, 15962)

In [15]:
import scipy.sparse
from scipy.sparse import hstack

# Classification

In [16]:
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

### K-Nearest Neighbours

In [17]:
KNN_classifier = KNeighborsClassifier(n_neighbors=5)
KNN_classifier.fit(X_train, train_labels)
KNN_prediction = KNN_classifier.predict(X_test)
KNN_score = KNN_classifier.score(X_test, test_labels)
KNN_f1 = f1_score(test_labels, KNN_prediction, average="weighted")

### Dummy Classifier

In [18]:
dummy_classifier = DummyClassifier(strategy="prior")
dummy_classifier.fit(X_train, train_labels)
dummy_prediction = dummy_classifier.predict(X_test)
dummy_score = dummy_classifier.score(X_test, test_labels)
dummy_f1 = f1_score(test_labels, dummy_prediction, average="weighted")

### Random Forest Classifier

In [19]:
RandomForest_classifier = RandomForestClassifier()
RandomForest_classifier.fit(X_train, train_labels)
RandomForest_prediction = RandomForest_classifier.predict(X_test)
RandomForest_score = RandomForest_classifier.score(X_test, test_labels)
RandomForest_f1 = f1_score(test_labels, RandomForest_prediction, average="weighted")

### Naive Bayes Classifier

In [20]:
NaiveBayes_classifier = GaussianNB()
NaiveBayes_classifier.fit(X_train.toarray(), train_labels)
NaiveBayes_prediction = NaiveBayes_classifier.predict(X_test)
NaiveBayes_score = NaiveBayes_classifier.score(X_test, test_labels)
NaiveBayes_f1 = f1_score(test_labels, NaiveBayes_prediction, average="weighted")

# Evaluation

In [21]:
print("Accuracy scores:\n KNN: {}\n Dummy: {}\n Random Forest: {}\n Naive Bayes: {}\n\n".format(KNN_score, dummy_score, RandomForest_score, NaiveBayes_score))
print("F1 scores:\n KNN: {}\n Dummy: {}\n Random Forest: {}\n Naive Bayes: {}\n\n".format(KNN_f1, dummy_f1, RandomForest_f1, NaiveBayes_f1))

Accuracy scores:
 KNN: 0.24
 Dummy: 0.16
 Random Forest: 0.36
 Naive Bayes: 0.24


F1 scores:
 KNN: 0.20444444444444443
 Dummy: 0.04413793103448277
 Random Forest: 0.34476190476190477
 Naive Bayes: 0.21087179487179486


